In [1]:
import pandas as pd 
import sklearn
import numpy as np
import nltk
import re
from io import StringIO # para armar las categorias

# Importamos paquetes a utilizar
from sklearn.feature_extraction.text import TfidfVectorizer
from unidecode import unidecode

# sklearn.feature_selection.chi2 para encontrar los términos que están más correlacionados
from sklearn.feature_selection import chi2

In [2]:
import os
'data_labeled.csv' in os.listdir()

True

In [3]:
df = pd.read_csv('data_labeled.csv', index_col='Unnamed: 0', encoding="latin1")

In [4]:
df = df[pd.notnull(df['message'])]
df.head()

date    comuna  \
0  2018-06-22 15:03:22  comuna01   
1  2018-06-22 14:54:07  comuna01   
2  2018-06-08 18:37:21  comuna01   
3  2018-06-07 20:37:10  comuna01   
4  2018-06-07 14:44:37  comuna01   

                                             message sentimiento      label  \
0  este pobre peaton pone en juego su seguridad p...    negativo    reclamo   
1  esto esta ocurriendo en la esquina de charcas ...    negativo    reclamo   
2  taller de teatro en fermin espacio cultural to...      neutro  propuesta   
3  hola querria saber como hay que hacer para asi...      neutro   consulta   
4  fijate como este sinverguenza cobra estacionam...    negativo    reclamo   

   categoria  
0  seguridad  
1  seguridad  
2    cultura  
3    cultura  
4   transito

## Preparación de texto, vectorización, stopwords y lemmatización

Lo primero que se realizará es eliminar todas las palabras que sean menor a dos letras.

In [5]:
def tres(doc):
    new_string = ' '.join([w for w in doc.split() if len(w)>2])  # eliminamos palabras menores a 3 letras
    return new_string

In [6]:
df['message'] = df['message'].apply(tres)

### Prepararemos un stopwords

In [7]:
stopwords = pd.read_csv('stopwords.csv',header=None)[0]

In [8]:
#a
stopwords = stopwords.str.replace(u"ã¡", "a")
stopwords = stopwords.str.replace('á', "a")

#e
stopwords = stopwords.str.replace(u"ã©", "e")
stopwords = stopwords.str.replace('é', "e")

#i
stopwords = stopwords.str.replace(u"ã­", "i")
stopwords = stopwords.str.replace('í', "i")

#o
stopwords = stopwords.str.replace(u"ã³", "o")
stopwords = stopwords.str.replace('ó', "o")

#u
stopwords = stopwords.str.replace(u"ãº", "u")
stopwords = stopwords.str.replace('ú', "u")

#ñ
stopwords = stopwords.str.replace(u"ã±","ñ")

### Lematización
Vamos a proceder a realizar una lemmatización de las palabras

In [9]:
lemmaDict = {}
with open('lemmatization-es.txt', 'rb') as f:
    # el lematizador viene con acentuación
    data = f.read().decode('utf8').replace(u'\r', u'').split(u'\n')
    data = [a.replace(u'á','a') for a in data]
    data = [a.replace(u'é','e') for a in data]
    data = [a.replace(u'í','i') for a in data]
    data = [a.replace(u'ó','o') for a in data]
    data = [a.replace(u'ú','o') for a in data]
    data = [a.split(u'\t') for a in data]
    # el lematizador viene con acentuación
    
for a in data:
    if len(a) >1:
        lemmaDict[a[1]] = a[0]

def lemmatize(word):
    return lemmaDict.get(word, word + u'*')
   
def test():
    for a in [ u'salio', u'usuarios', u'abofeteeis', u'diferenciando', u'diferenciandola' ]:
        print(lemmatize(a))
test()

salir
usuario
abofetear
diferenciar
diferenciandola*


Los clasificadores y algoritmos de aprendizaje no pueden procesar directamente los documentos de texto en su forma original, ya que la mayoría de ellos espera vectores de características numéricas con un tamaño fijo en lugar de los documentos de texto sin formato con longitud variable. Por lo tanto, durante el paso de preproceso, los textos se convierten a una representación más manejable.

Un enfoque común para extraer características del texto es usar el modelo de la bolsa de palabras: un modelo donde, para cada documento, una narrativa de queja en nuestro caso, se tiene en cuenta la presencia (y, a menudo, la frecuencia) de las palabras, pero el orden que ocurren se ignora.

Específicamente, para cada término en nuestro conjunto de datos, calcularemos una medida denominada Frecuencia de términos, Frecuencia de documentos inversa, abreviada a tf-idf. Utilizaremos sklearn.feature_extraction.text.TfidfVectorizer para calcular un vector tf-idf para cada una de las narrativas de quejas del consumidor:

* **sublinear_df** se establece en True para usar una forma logarítmica para la frecuencia.

* **min_df** es el número mínimo de documentos que una palabra debe estar presente para mantenerse.

* **norm** se establece en l2 , para asegurar que todos nuestros vectores de características tengan una norma euclidiana de 1.

* **ngram_range** se establece en (1, 2) para indicar que queremos considerar los unigramas y los bigrams.

* **stop_words** está establecido en "spanish" para eliminar todos los pronombres comunes ( "a" , "el" , ...) para reducir la cantidad de características ruidosas.

In [10]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2, norm='l2', encoding='latin1', ngram_range=(1, 2), stop_words = list(stopwords), preprocessor=lemmatize)

features = tfidf.fit_transform(df['message']).toarray()

In [11]:
features.shape

(278, 1717)

In [12]:
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Codificando cada una de las columnas y generando bases

In [13]:
df_sentiment = df[['message', 'sentimiento']]
df_sentiment.columns = ['message', 'sentimiento']

df_sentiment['sentiment_id'] = df_sentiment['sentimiento'].factorize()[0]
sentiment_id_df = df_sentiment[['sentimiento', 'sentiment_id']].drop_duplicates().sort_values('sentiment_id')

sentiment_to_id = dict(sentiment_id_df.values)
id_to_sentiment = dict(sentiment_id_df[['sentiment_id', 'sentimiento']].values)

df_sentiment.head()

/Users/hernangoldenberg/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


message sentimiento  sentiment_id
0  este pobre peaton pone juego seguridad para cr...    negativo             0
1  esto esta ocurriendo esquina charcas billinguh...    negativo             0
2  taller teatro fermin espacio cultural todos lo...      neutro             1
3  hola querria saber como hay que hacer para asi...      neutro             1
4  fijate como este sinverguenza cobra estacionam...    negativo             0

In [14]:
df_label = df[['message', 'label']]
df_label.columns = ['message', 'label']

df_label['label_id'] = df_label['label'].factorize()[0]
label_id_df = df_label[['label', 'label_id']].drop_duplicates().sort_values('label_id')

label_to_id = dict(label_id_df.values)
id_to_label = dict(label_id_df[['label_id', 'label']].values)

df_label.head()

/Users/hernangoldenberg/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


message      label  label_id
0  este pobre peaton pone juego seguridad para cr...    reclamo         0
1  esto esta ocurriendo esquina charcas billinguh...    reclamo         0
2  taller teatro fermin espacio cultural todos lo...  propuesta         1
3  hola querria saber como hay que hacer para asi...   consulta         2
4  fijate como este sinverguenza cobra estacionam...    reclamo         0

In [15]:
df_category = df[['message', 'categoria']]
df_category.columns = ['message', 'categoria']

df_category['category_id'] = df_category['categoria'].factorize()[0]
category_id_df = df_category[['categoria', 'category_id']].drop_duplicates().sort_values('category_id')

category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'categoria']].values)

df_category.head()

/Users/hernangoldenberg/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


message  categoria  category_id
0  este pobre peaton pone juego seguridad para cr...  seguridad            0
1  esto esta ocurriendo esquina charcas billinguh...  seguridad            0
2  taller teatro fermin espacio cultural todos lo...    cultura            1
3  hola querria saber como hay que hacer para asi...    cultura            1
4  fijate como este sinverguenza cobra estacionam...   transito            2

Se puede usar sklearn.feature_selection.chi2 para encontrar los términos que están más correlacionados con cada uno de los productos:

In [38]:
# Análisis para Sentimiento
labels_sent = df_sentiment['sentimiento']
N = 2
for sentimiento, sentiment_id in sorted(sentiment_to_id.items()):
    features_chi2 = chi2(features, labels_sent.index == sentiment_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(sentimiento))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'negativo':
  . Most correlated unigrams:
. espacios
. juego
  . Most correlated bigrams:
. lunes viernes
. buenos aire
# 'neutro':
  . Most correlated unigrams:
. charcas
. ocurriendo
  . Most correlated bigrams:
. buenos aire
. bien gracias
# 'positivo':
  . Most correlated unigrams:
. culpina
. fermin
  . Most correlated bigrams:
. fermin espacio
. culpina venite


In [37]:
# Análisis para label
labels_label = df_label['label']
N = 2
for label, label_id in sorted(label_to_id.items()):
    print()
    features_chi2 = chi2(features, labels_label.index == label_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(label))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    


# 'consulta':
  . Most correlated unigrams:
. culpina
. fermin
  . Most correlated bigrams:
. fermin espacio
. culpina venite

# 'propuesta':
  . Most correlated unigrams:
. charcas
. ocurriendo
  . Most correlated bigrams:
. buenos aire
. bien gracias

# 'reclamo':
  . Most correlated unigrams:
. espacios
. juego
  . Most correlated bigrams:
. lunes viernes
. buenos aire


In [39]:
# Análisis para categoria
labels_category = df_category['categoria']
N = 2
for categoria, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels_category.index == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(categoria))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'abl':
  . Most correlated unigrams:
. realizan
. visitas
  . Most correlated bigrams:
. buenos aire
. saber hacer
# 'cultura':
  . Most correlated unigrams:
. charcas
. ocurriendo
  . Most correlated bigrams:
. buenos aire
. bien gracias
# 'gobierno':
  . Most correlated unigrams:
. comprobante
. cobra
  . Most correlated bigrams:
. lunes viernes
. buenos aire
# 'mascotas':
  . Most correlated unigrams:
. nota
. sectores
  . Most correlated bigrams:
. bien gracias
. comuna transito
# 'plazas':
  . Most correlated unigrams:
. responsabilidad
. bustamante
  . Most correlated bigrams:
. baldosas rota
. sanchez bustamante
# 'seguridad':
  . Most correlated unigrams:
. espacios
. juego
  . Most correlated bigrams:
. lunes viernes
. buenos aire
# 'situacion_de_calle':
  . Most correlated unigrams:
. ocurriendo
. sinverguenza
  . Most correlated bigrams:
. sres comuna
. estacionamiento debe
# 'transito':
  . Most correlated unigrams:
. culpina
. fermin
  . Most correlated bigrams:
. fermin

** Como puede observarse no da nada bien **

In [19]:
tfidf.get_feature_names()

['abajo',
 'abandonada',
 'abandonados',
 'abl',
 'abl pagamosno',
 'abl tampoco',
 'abrazo',
 'absolutamente',
 'absolutamente dia',
 'aca',
 'acabamos',
 'acabamos crear',
 'acabo',
 'accesos',
 'accesos tampco',
 'accidente',
 'accidentes',
 'acosta',
 'actividad',
 'actora',
 'actoral',
 'actoral antonin',
 'acumula',
 'acumula agua',
 'ademas',
 'ademas contaminacion',
 'ademas posible',
 'adjunto',
 'adjunto foto',
 'adjunto fotos',
 'adonde',
 'adonde debo',
 'adopciones',
 'adopciones perdidos',
 'adoquinado',
 'adultos',
 'afuera',
 'agradecemos',
 'agradecer',
 'agradecido',
 'agradecido atencion',
 'agradezco',
 'agua',
 'agustin',
 'agustin fox',
 'ahi',
 'ahora',
 'aire',
 'aire horacio',
 'alambres',
 'alberdi',
 'alcohol',
 'algogracias',
 'alguien',
 'alguien corte',
 'alguien puede',
 'algun',
 'algun servicio',
 'alguna',
 'alguna autoridad',
 'alguna medida',
 'alguna vez',
 'alli',
 'alrededor',
 'altas',
 'altas horas',
 'altura',
 'amarillo',
 'ambas',
 'ambiental

Guardamos las bases codificadas

In [20]:
df_sentiment.to_csv('df_sentiment.csv')
df_label.to_csv('df_label.csv')
df_category.to_csv('df_category.csv')